<a href="https://colab.research.google.com/github/mabela1/Sentiment-Analysis-with-BERT-Neural-Network/blob/main/Sentiment_Analysis_with_BERT_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and Import Dependencies

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
import numpy as np
import pandas as pd

Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Encode and Calculate Sentiment

In [ ]:
tokens = tokenizer.encode('It was super!', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-2.0860, -2.1455, -0.4978,  1.3687,  2.6930]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

5

Collect Reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

Load Reviews into DataFrame and Score

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df['review'].iloc[3]

'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.'

In [ ]:
df

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,Great staff and food. Must try is the pan fri...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5
3,I came to Social brew cafe for brunch while ex...,5
4,It was ok. The coffee wasn't the best but it w...,3
5,We came for brunch twice in our week-long visi...,4
6,I went here a little while ago- a beautiful mo...,2
7,Ron & Jo are on the go down under and Wow! We...,5
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4
